In [2]:
import pandas
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

In [3]:
#load data into memory
data3 = np.genfromtxt('data3.dat', delimiter='  ')
data3 = data3.transpose()

#test to see if other data works
drawsml = np.genfromtxt('drawsml.dat', delimiter='  ')
drawsgmm = np.genfromtxt('drawsgmm.dat', delimiter='  ')

In [4]:
def sim_ll(y,x1,x2,t1,t2,sigma,v=False):
    """Simulate the log likelihood function by integrating out"""
    #hard coded draws
    eps1 = np.genfromtxt('drawsml.dat', delimiter='  ').transpose()
    sim_like = 0 
    #set up vectors for adding
    zeros = np.full(len(x1),.0001)
    const = np.full(len(x1), 100)   
    
    #run simulations, to simulate likelihoods
    for s in range(len(eps1)): 
        #calculate f inverse
        mc1 = np.exp(t1 + t2*x1 + sigma*eps1[s])
        safe_log1 = np.maximum( 3*y - const - mc1 ,zeros)
        f_inv =  (np.log(safe_log1) - t1 - t2*x2)
        safe_f_inv = np.maximum( f_inv ,zeros)
        #calculate f prime
        f_prime = (3.0/sigma)*1/(3.0*y - const - mc1)
        sim_like = sim_like + norm(0,sigma).pdf(f_inv)*f_prime
    
    #finalized simulated likelihoods
    safe_sim = np.maximum(sim_like/len(eps1),zeros)    
    return np.log(safe_sim).sum()


class part_b(GenericLikelihoodModel):
    """class for evaluating question 3 part b"""
    
    def nloglikeobs(self, params):
        t1, t2, sigma = params
        y = self.endog
        x1,x2 = self.exog.transpose()
        return -sim_ll(y, x1,x2,t1,t2,sigma)

    
    def fit(self, start_params=None, maxiter=100, maxfun=5000, **kwds):
        if start_params == None:
            start_params = np.array([1,1,1])
        return super(part_b, self).fit(start_params=start_params,
                                       maxiter=maxiter, maxfun=maxfun, **kwds)

    
model_b = part_b(data3[0],data3[1:].transpose()).fit(start_params = [1,1,1])
print(model_b.summary(xname=['theta_1','theta_2','sigma']))

Optimization terminated successfully.
         Current function value: -3.174360
         Iterations: 92
         Function evaluations: 166
                                part_b Results                                
Dep. Variable:                      y   Log-Likelihood:                 158.72
Model:                         part_b   AIC:                            -313.4
Method:            Maximum Likelihood   BIC:                            -309.6
Date:                Fri, 28 Sep 2018                                         
Time:                        09:11:33                                         
No. Observations:                  50                                         
Df Residuals:                      48                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

In [8]:
def sim_g(t1,t2,sigma,y,x1,x2,eps1,eps2):
    """simulate moment conditions, given predetermined
    simulations"""
    g = 0
    for s in range(len(eps1)):
        y_hat = 1/3.0*(100 + np.exp(t1 + t2*x1 + sigma*eps1[s])
                       + np.exp(t1 + t2*x2 + sigma*eps2[s]))
        g = g + (y - y_hat)
        
    return g/ (len(eps1)*len(eps2))


def sim_g_sq(t1,t2,sigma,y,x1,x2,eps1,eps2):
    """simulate square moment conditions, given predetermined
    simulations"""
    g = 0
    for s in range(len(eps1)):
        y_hat = 1/3.0*(100 + np.exp(t1 + t2*x1 + sigma*eps1[s])
                       + np.exp(t1 + t2*x2 + sigma*eps2[s]))
        g = g + (y**2-y_hat**2)
    return g/(len(eps1)*len(eps2))


class part_d(GMM):
    """class for evaluating question 3 part d"""

    def __init__(self, *args, **kwds):
        """set appropriate counts for moment conditions and parameters"""
        kwds.setdefault('k_moms', 4)
        kwds.setdefault('k_params',3)
        super(part_d, self).__init__(*args, **kwds)
    
    
    def fit(self, start_params=None, maxiter=10000, **kwds):
        if start_params == None:
            start_params = np.array([.5, .5, .5])
        return super(part_d, self).fit(start_params=start_params,
                                       maxiter=maxiter,  **kwds)

        
    def momcond(self, params):
        t1,t2, sigma = params   
        y = self.endog
        x1,x2 = self.exog.transpose() 
        
        #initialize draws
        eps = np.genfromtxt('drawsgmm.dat', delimiter='  ').transpose()
        eps1 = eps[0:len(eps)/2]
        eps2 = eps[len(eps)/2:len(eps)]
        
        #calc moment conditions
        g = sim_g(t1,t2,sigma,y,x1,x2,eps1,eps2)
        g_sq = sim_g_sq(t1,t2,sigma,y,x1,x2,eps1,eps2)
        return np.column_stack((g,g*x1,g*x2,g_sq))


model_d = part_d(data3[0],data3[1:].transpose(), None)


#print first iteration (i.e. 1 stage)
result_d_stage1 = model_d.fit(maxiter=1, optim_method='nm', wargs=dict(centered=False))
print(result_d_stage1.summary(xname=['theta_1','theta_2','sigma']))
print('\n')


#print second iteration (i.e. 2 stage)
result_d_stage2 = model_d.fit(maxiter=2, optim_method='nm', wargs=dict(centered=False))
print(result_d_stage2.summary(xname=['theta_1','theta_2','sigma']))

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 42
         Function evaluations: 84
                                part_d Results                                
Dep. Variable:                      y   Hansen J:                    1.552e-05
Model:                         part_d   Prob (Hansen J):                 0.997
Method:                           GMM                                         
Date:                Fri, 28 Sep 2018                                         
Time:                        09:14:08                                         
No. Observations:                  50                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
theta_1        0.8812      0.025     34.668      0.000       0.831       0.931
theta_2        0.2548      0.220      1.160      0.246      -0.176      